In [2]:
import io
import requests
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig
import os
os.environ['HF_HOME'] = '/netscratch/kadir/xl-vlms/cache'



/root/anaconda3/envs/xl_vlm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# step 1: Setup constant
device = "cuda"
dtype = torch.float16



In [3]:
os.environ['HF_HOME'] = '/netscratch/kadir/xl-vlms/cache/hub'
processor = AutoProcessor.from_pretrained("StanfordAIMI/CheXagent-8b", cache_dir = '/netscratch/kadir/xl-vlms/cache/hub',  trust_remote_code=True)
generation_config = GenerationConfig.from_pretrained("StanfordAIMI/CheXagent-8b")
model = AutoModelForCausalLM.from_pretrained("StanfordAIMI/CheXagent-8b", cache_dir = '/netscratch/kadir/xl-vlms/cache/hub', torch_dtype=dtype, trust_remote_code=True)


Loading checkpoint shards: 100%|██████████| 7/7 [02:34<00:00, 22.12s/it]


In [4]:
from torchinfo import summary
print(summary(model, depth=3))

ModuleNotFoundError: No module named 'torchinfo'

In [7]:
for name, module in model.named_modules():
    print(name, )


vision_model
vision_model.embeddings
vision_model.embeddings.patch_embedding
vision_model.encoder
vision_model.encoder.layers
vision_model.encoder.layers.0
vision_model.encoder.layers.0.self_attn
vision_model.encoder.layers.0.self_attn.dropout
vision_model.encoder.layers.0.self_attn.qkv
vision_model.encoder.layers.0.self_attn.projection
vision_model.encoder.layers.0.layer_norm1
vision_model.encoder.layers.0.mlp
vision_model.encoder.layers.0.mlp.activation_fn
vision_model.encoder.layers.0.mlp.fc1
vision_model.encoder.layers.0.mlp.fc2
vision_model.encoder.layers.0.layer_norm2
vision_model.encoder.layers.1
vision_model.encoder.layers.1.self_attn
vision_model.encoder.layers.1.self_attn.dropout
vision_model.encoder.layers.1.self_attn.qkv
vision_model.encoder.layers.1.self_attn.projection
vision_model.encoder.layers.1.layer_norm1
vision_model.encoder.layers.1.mlp
vision_model.encoder.layers.1.mlp.activation_fn
vision_model.encoder.layers.1.mlp.fc1
vision_model.encoder.layers.1.mlp.fc2
visio

In [1]:
import io
import requests
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig

# Step 1: Setup constants
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16
cache_dir = "/netscratch/kadir/xl-vlms/cache/hub"  # Your custom cache directory

# Step 2: Load Processor and Model with cache_dir
processor = AutoProcessor.from_pretrained(
    "StanfordAIMI/CheXagent-8b", 
    cache_dir=cache_dir, 
    trust_remote_code=True
)
generation_config = GenerationConfig.from_pretrained(
    "StanfordAIMI/CheXagent-8b", 
    cache_dir=cache_dir
)
model = AutoModelForCausalLM.from_pretrained(
    "StanfordAIMI/CheXagent-8b", 
    cache_dir=cache_dir, 
    torch_dtype=dtype, 
    trust_remote_code=True
).to(device)



/root/anaconda3/envs/xl_vlm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 7/7 [01:41<00:00, 14.46s/it]


In [2]:
print(model)

CheXagentForConditionalGeneration(
  (vision_model): CheXagentVisionModel(
    (embeddings): CheXagentVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): CheXagentEncoder(
      (layers): ModuleList(
        (0-39): 40 x CheXagentEncoderLayer(
          (self_attn): CheXagentAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): CheXagentMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (

In [4]:
import torch
from PIL import Image

# Step 1: Load and prepare the image
image_path = "/home/kadir/xl-vlms/playground/images/synpic32933.jpg"
images = [Image.open(image_path).convert("RGB")]

# Step 2: Define the prompt and prepare inputs
prompt = f'Describe "Airway" in 10 sentaance'
inputs = processor(
    images=images, 
    text=f" USER: <s>{prompt} ASSISTANT: <s>", 
    return_tensors="pt"
).to(device=device, dtype=dtype)

# Show the input shapes
print(f"The image shape is: {inputs['pixel_values'].shape}")
print(f"The language shape is: {inputs['input_ids'].shape}")

# Step 3: Initialize hooks and storage for layer output shapes and weights
output_shapes = {}  # Dictionary to store layer output shapes
weights = {}        # Dictionary to store weight shapes and sizes

# Hook function to capture output shapes during the forward pass
def hook_fn(module, input, output):
    # Record the output shape of the module
    output_shapes[module] = output.shape if isinstance(output, torch.Tensor) else "Non-Tensor Output"

# Step 4: Register hooks and collect layer weight information
hooks = []
for name, module in model.named_modules():
    # Register hook for each layer to capture the output shape
    hooks.append(module.register_forward_hook(hook_fn))
    
    # Collect the weight shapes for each layer
    layer_weights = []
    for param in module.parameters():
        layer_weights.append(list(param.shape))  # Record parameter shapes
    weights[name] = layer_weights if layer_weights else "No Weights"

# Step 5: Perform a forward pass with the model and generate the response
print(generation_config)
#generation_config.num_beams =1
with torch.no_grad():
    output = model.generate(**inputs, generation_config=generation_config)

# Decode and print the generated response
response = processor.tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Generated Response:\n{response}")

# Step 6: Print the layer information (name, weight shapes, and output shape)
print("\nLayer Information (Name, Weight Shapes, Output Shape):\n")
print(f"{'Layer Name':<60} {'Weight Shapes':<40} {'Output Shape'}")
print("-" * 120)

# Filter and print only the relevant layers for analysis
layers_of_interest = ["vision_model.embeddings.patch_embedding","vision_model.encoder.layers.39.mlp.fc2", "vision_model.post_layernorm", "language_model.model.embed_tokens", "qformer.encoder.layer.11.output_query.dense", "language_projection",  "language_model.model.embed_tokens", "language_model.model.layers.31","language_model.lm_head"]
for module, shape in output_shapes.items():
    # Get the layer name by searching in the model's named modules
    layer_name = next((name for name, m in model.named_modules() if m == module), "Unknown")
    
    # If the layer is one of the layers of interest, print its info
    if layer_name in layers_of_interest:
        weight_shapes = str(weights[layer_name]) if weights[layer_name] != "No Weights" else "No Weights"
        print(f"{layer_name:<60} {weight_shapes:<40} {shape}")

# Clean up hooks to avoid memory leaks
for hook in hooks:
    hook.remove()


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The image shape is: torch.Size([1, 1, 3, 448, 448])
The language shape is: torch.Size([1, 25])
GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_length": 512,
  "num_beams": 5
}

Generated Response:
The hilar contours are normal.

Layer Information (Name, Weight Shapes, Output Shape):

Layer Name                                                   Weight Shapes                            Output Shape
------------------------------------------------------------------------------------------------------------------------
vision_model.embeddings.patch_embedding                      [[1408, 3, 14, 14], [1408]]              torch.Size([1, 1408, 32, 32])
vision_model.encoder.layers.39.mlp.fc2                       [[1408, 6144], [1408]]                   torch.Size([1, 1025, 1408])
vision_model.post_layernorm                                  [[1408], [1408]]                         torch.Size([1, 1408])
qformer.encoder.layer.11.output_query.dense                  [[768, 3072]

qformer outptut = (128, 768)
language_projecton = (128, 4096)
text_embedding = (numbe_of_tokens, 4096)
inputs_embeds = torch.cat([input_vis, inputs_lang], dim=1) shape (batch,numbe_of_tokens+128, 4096 )
language_model.model.layers.31/language_model.model.norm = [1, 128+num_token, 4096]

output = [number of ouput token]
# step 5: conditioned on the images and/or prompts
        if pixel_values is not None:
            inputs_embeds = torch.cat([input_vis, inputs_lang], dim=1)
            attention_mask = torch.cat([vis_atts, lang_atts], dim=1)
        else:
            inputs_embeds = inputs_lang
            attention_mask = lang_atts

        outputs = self.language_model.generate(
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            **generate_kwargs,
        )

In [24]:
image_path = "/home/kadir/xl-vlms/playground/images/synpic32933.jpg"
images = [Image.open(image_path).convert("RGB")]
print(model)

CheXagentForConditionalGeneration(
  (vision_model): CheXagentVisionModel(
    (embeddings): CheXagentVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): CheXagentEncoder(
      (layers): ModuleList(
        (0-39): 40 x CheXagentEncoderLayer(
          (self_attn): CheXagentAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): CheXagentMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (

In [5]:
prompt = f'Describe "Airway", '
inputs = processor(images=images, text=f" USER: <s>{prompt} ASSISTANT: <s>", return_tensors="pt").to(device=device, dtype=dtype)
model = model.to(device)
output = model.generate(**inputs, generation_config=generation_config)[0]
response = processor.tokenizer.decode(output, skip_special_tokens=True)

/root/.cache/huggingface/modules/transformers_modules/StanfordAIMI/CheXagent-8b/4934e91451945c8218c267aae9c34929a7677829/processing_chexagent.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(pixel_values) for pixel_values in encoding_image_processor["pixel_values"]]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [6]:
print(response)

The hilar contours are normal.


In [7]:
from typing import Any, Callable, Dict

import numpy as np
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig

from models.image_text_model import ImageTextModel
import logging

__all__ = ["Molmo"]


class Molmo(ImageTextModel):

    def set_model(
        self,
    ) -> None:
        logging.debug("This is a debug message (useful for detailed troubleshooting).")
        self.model_ = AutoModelForCausalLM.from_pretrained(
            self.model_name_or_path,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
            local_files_only=self.local_files_only,
        )

    def get_model(
        self,
    ) -> Callable:

        return self

    def get_language_model(
        self,
    ) -> Callable:

        return self.model_.model.transformer

    def get_lm_head(
        self,
    ) -> Callable:

        return self.model_.model.transformer.ff_out

    def set_processor(
        self,
    ) -> None:

        self.processor_ = AutoProcessor.from_pretrained(
            self.processor_name,
            local_files_only=self.local_files_only,
            trust_remote_code=True,
            torch_dtype=torch.float16,
        )
        self.tokenizer_ = self.processor_.tokenizer

    def set_preprocessor(
        self,
    ) -> None:

        self.preprocessor_ = self.preprocess_input

    def get_conversation_template(
        self,
        instruction: str = "What are these?",
        response: str = "",
        **kwargs: Any,
    ) -> Dict[str, Any]:

        conversation = instruction
        if response:
            conversation += f" Answer: {response}"
        return conversation

    def preprocess_input(
        self,
        instruction: str = "What are these?",
        image_file: str = None,
        response: str = "",
        **kwargs: Any,
    ) -> Dict[str, Any]:

        text = self.get_conversation_template(
            instruction=instruction,
            response=response,
            image_file=image_file,
        )

        image = Image.open(image_file).convert("RGB")
        image = np.array(image)

        inputs = self.processor_.process(
            text=text,
            images=[image],
            padding=True,
            return_tensors="pt",
        )

        return inputs

    def preprocessor(
        self,
        instruction: str = "What are these?",
        image_file: str = "",
        response: str = "",
        generation_mode: bool = False,
        **kwargs: Any,
    ):
        preprocessor = self.get_preprocessor()
        inputs = preprocessor(
            instruction=instruction,
            image_file=image_file,
            response=response,
            generation_mode=generation_mode,
        )
        return inputs

    def generate(
        self,
        max_new_tokens: int = 200,
        do_sample: bool = False,
        **inputs: Dict[str, Any],
    ):
        inputs = {k: v.unsqueeze(0).to(self.model_.device) for k, v in inputs.items()}
        device_type = "cuda" if torch.cuda.is_available() else "cpu"
        with torch.autocast(
            device_type=device_type, enabled=True, dtype=self.model_.dtype
        ):
            output = self.model_.generate_from_batch(
                inputs,
                GenerationConfig(
                    max_new_tokens=max_new_tokens,
                    stop_strings="<|endoftext|>",
                    do_sample=do_sample,
                ),
                tokenizer=self.tokenizer_,
            )
        return output





In [28]:
from typing import Any, Callable, Dict, List

import torch
from qwen_vl_utils import process_vision_info
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration

from models.image_text_model import ImageTextModel

__all__ = ["ChexAgent"]


class ChexAgent(ImageTextModel):

    def set_model(
        self,
    ) -> None:

        self.model_ = AutoModelForCausalLM.from_pretrained(
            self.model_name_or_path,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
            cache_dir ='/netscratch/kadir/xl-vlms/cache/hub',
            local_files_only=self.local_files_only,
        )


    def get_language_model(
        self,
    ) -> Callable:

        return self.model_.model

    def get_lm_head(
        self,
    ) -> Callable:

        return self.model_.lm_head

    def set_processor(
        self,
    ) -> None:

        self.processor_ = AutoProcessor.from_pretrained(
            self.processor_name,
            local_files_only=self.local_files_only,
        )
        self.tokenizer_ = self.processor_.tokenizer

    def set_preprocessor(
        self,
    ) -> None:

        self.preprocessor_ = self.preprocess_input

    def get_conversation_round(
        self,
        instruction: str = "What are these?",
        response: str = "",
        image_file: str = "",
    ) -> List[Dict[str, Any]]:

        conversation = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "image": image_file,
                    },
                    {"type": "text", "text": instruction},
                ],
            },
        ]
        if response:
            conversation.append(
                {
                    "role": "assistant",
                    "content": [
                        {"type": "text", "text": response},
                    ],
                },
            )

        return conversation

    def get_conversation_template(
        self,
        instruction: str = "What are these?",
        response: str = "",
        image_file: str = "",
        **kwargs: Any,
    ) -> Dict[str, Any]:

        conversation = self.get_conversation_round(
            instruction=instruction,
            response=response,
            image_file=image_file,
        )
        return conversation

    def preprocess_text(
        self,
        conversation,
        generation_mode: bool = False,
        **kwargs: Any,
    ) -> str:

        prompt = self.processor_.apply_chat_template(
            conversation,
            add_generation_prompt=generation_mode,
            tokenize=False,
        )

        return prompt

    def preprocess_images(
        self,
        conversation,
        **kwargs: Any,
    ) -> List:

        image_inputs, video_inputs = process_vision_info(conversation)

        return image_inputs, video_inputs

    def preprocess_input(
        self,
        instruction: str = "What are these?",
        image_file: str = None,
        response: str = "",
        generation_mode: bool = False,
        **kwargs: Any,
    ) -> Dict[str, Any]:

        conversation = self.get_conversation_template(
            instruction=instruction,
            response=response,
            image_file=image_file,
        )

        image, video = self.preprocess_images(conversation)
        text = self.preprocess_text(
            conversation,
            generation_mode=generation_mode,
        )

        inputs = self.processor_(
            text=[text],
            images=image,
            videos=video,
            padding=True,
            return_tensors="pt",
        )

        return inputs


In [30]:

model_name= "StanfordAIMI/CheXagent-8b"

model_class = ChexAgent(
            model_name_or_path=model_name,
            cache_dir = '/netscratch/kadir/xl-vlms/cache/hub',
            processor_name=model_name,
            local_files_only=False,
            trust_remote_code=True,
        
        )

Loading checkpoint shards: 100%|██████████| 7/7 [00:12<00:00,  1.76s/it]


In [2]:
import torch
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import matplotlib
print("Matplotlib version for info:", matplotlib.__version__)

Matplotlib version for info: 3.9.4


In [15]:
results_dict = torch.load('/netscratch/kadir/xl-vlms/features/save_hidden_states_for_token_of_interest_chexagent_A_generation_split_train.pth')
#print(results_dict['model_predictions'])

print(torch.tensor(results_dict['token_of_interest_mask']).sum())

tensor(10368)
